In [216]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [217]:
#load in datasets
import pandas as pd

train_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
train_df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [218]:
#drop rows where the missing values are greater than half of the length of the df, in the train&test df
to_drop = list(train_df.columns[train_df.isna().sum()>(len(train_df)/2)])
train_df.drop(to_drop, axis=1, inplace=True)
test_df.drop(to_drop, axis=1, inplace=True)

In [219]:
to_drop = list(test_df.columns[test_df.isna().sum()>(len(test_df)/2)])
train_df.drop(to_drop, axis=1, inplace=True)
test_df.drop(to_drop, axis=1, inplace=True)

In [220]:
#fill every missing values where the type is int or float
num_cols_train = train_df.select_dtypes(include=['int', 'float']).columns.tolist()
num_cols_test = test_df.select_dtypes(include=['int', 'float']).columns.tolist() #doesnt has the saleprice column

In [221]:
train_df[num_cols_test] = train_df[num_cols_test].fillna(train_df[num_cols_test].mean())
test_df[num_cols_test] = test_df[num_cols_test].fillna(test_df[num_cols_test].mean())

In [222]:
#check how many columns have missing data
len(list(train_df.columns[train_df.isna().sum()>0]))

11

In [223]:
#try and concat the two dfs and then replace all categorical columns with numbers
#because if you dont concat the two together, you get a different number of columns in each df
final = pd.concat([train_df, test_df], axis=0)
obj_cols_fin = final.select_dtypes(include=['object']).columns.tolist()
final = pd.get_dummies(final, columns=obj_cols_fin)

In [224]:
final.shape

(2919, 272)

In [225]:
#remove duplicate columns
final = final.loc[:,~final.columns.duplicated()]

In [226]:
#get all the categorical data and turn into numbers
obj_cols_train = train_df.select_dtypes(include=['object']).columns.tolist()
train_df = pd.get_dummies(train_df, columns=obj_cols_train)
obj_cols_test = test_df.select_dtypes(include=['object']).columns.tolist()
test_df = pd.get_dummies(test_df, columns=obj_cols_test)

In [227]:
print(train_df.shape)
print(test_df.shape)

(1460, 272)
(1459, 255)


In [228]:
#test if the shape of the "final" df is correct
final.shape

(2919, 272)

In [229]:
from copy import deepcopy

#divide final df into train sub
train_sub = deepcopy(final.iloc[:1460,:])
train_sub

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706.0,...,0,0,0,1,0,0,0,0,1,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978.0,...,0,0,0,1,0,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486.0,...,0,0,0,1,0,0,0,0,1,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216.0,...,0,0,0,1,1,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655.0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,62.0,7917,6,5,1999,2000,0.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1456,1457,20,85.0,13175,6,6,1978,1988,119.0,790.0,...,0,0,0,1,0,0,0,0,1,0
1457,1458,70,66.0,9042,7,9,1941,2006,0.0,275.0,...,0,0,0,1,0,0,0,0,1,0
1458,1459,20,68.0,9717,5,6,1950,1996,0.0,49.0,...,0,0,0,1,0,0,0,0,1,0


In [230]:
#fill misssing values
#train_sub = train_sub.fillna(train_sub.mean())
#set input X and y
X_train, y_train = train_sub.drop(["SalePrice"],axis=1).to_numpy(), train_sub["SalePrice"].to_numpy()

In [231]:
X_train.shape

(1460, 271)

In [232]:
#get LR model
from sklearn.linear_model import LinearRegression
#train model
lr = LinearRegression().fit(X_train, y_train)

In [233]:
#train_sub["predictions"] = lr.predict(X_train)
#train_sub

In [234]:
#get the other half of the final df to be the test sub df
test_sub = deepcopy(final.iloc[1460:,:].drop(["SalePrice"],axis=1))
test_sub

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1461,20,80.0,11622,5,6,1961,1961,0.0,468.0,...,0,0,0,1,0,0,0,0,1,0
1,1462,20,81.0,14267,6,6,1958,1958,108.0,923.0,...,0,0,0,1,0,0,0,0,1,0
2,1463,60,74.0,13830,5,5,1997,1998,0.0,791.0,...,0,0,0,1,0,0,0,0,1,0
3,1464,60,78.0,9978,6,6,1998,1998,20.0,602.0,...,0,0,0,1,0,0,0,0,1,0
4,1465,120,43.0,5005,8,5,1992,1992,0.0,263.0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,21.0,1936,4,7,1970,1970,0.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1455,2916,160,21.0,1894,4,5,1970,1970,0.0,252.0,...,0,0,0,1,1,0,0,0,0,0
1456,2917,20,160.0,20000,5,7,1960,1996,0.0,1224.0,...,0,0,0,1,1,0,0,0,0,0
1457,2918,85,62.0,10441,5,5,1992,1992,0.0,337.0,...,0,0,0,1,0,0,0,0,1,0


In [235]:
#convert to numpy
X_test = test_sub.to_numpy()
X_test.shape

(1459, 271)

In [236]:
X_train.shape

(1460, 271)

In [237]:
#test_df["SalePrice"] = lr.predict(X_test)
#test_df

In [238]:
#test_df[["Id","SalePrice"]].to_csv("prediction1.csv", index=False)

In [239]:
import xgboost
regressor = xgboost.XGBRegressor(n_estimators=500)

In [240]:
#define parameter grid for hyperparameter tuning
booster=['gbtree','gblinear']
base_score=[0.25,0.5,0.75,1]

n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5, 10, 15]
booster=['gbtree','gblinear']
learning_rate=[0.05,0.1,0.15,0.20]
min_child_weight=[1,2,3,4]


# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
    }

In [78]:
# Set up the random search with 4-fold cross validation
from sklearn.model_selection import RandomizedSearchCV
random_cv = RandomizedSearchCV(estimator=regressor,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=50,
            scoring = 'neg_mean_absolute_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)

In [79]:
#print(X_test.shape)
#print(X_train.shape)

(1459, 271)
(1460, 271)


In [80]:
#fit the random_cv 
random_cv.fit(X_train,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[07:32:09] WARNING: ../src/learner.cc:767: 
Parameters: { "max_depth", "min_child_weight" } are not used.

[CV 4/5] END base_score=0.75, booster=gblinear, learning_rate=0.2, max_depth=15, min_child_weight=1, n_estimators=1500;, score=(train=-13607.662, test=-16532.695) total time=   6.4s
[CV 1/5] END base_score=1, booster=gbtree, learning_rate=0.1, max_depth=3, min_child_weight=4, n_estimators=1100;, score=(train=-2084.147, test=-15491.192) total time=  17.9s
[CV 5/5] END base_score=1, booster=gbtree, learning_rate=0.1, max_depth=3, min_child_weight=4, n_estimators=1100;, score=(train=-2105.945, test=-16387.096) total time=  18.1s
[CV 4/5] END base_score=0.5, booster=gbtree, learning_rate=0.05, max_depth=15, min_child_weight=4, n_estimators=900;, score=(train=-4.455, test=-14660.554) total time= 1.1min
[CV 3/5] END base_score=0.25, booster=gbtree, learning_rate=0.15, max_depth=2, min_child_weight=3, n_estimators=1500;, score=(train=-3113.506, test=-16141.913) total time=  17.5s
[07:34:

RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, gpu_id=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                          predictor=None, random_state=None, ...),
                   n_iter=50, n_jobs=4,
                   param_distributions={'base_score': [0.25, 0.5, 0.75, 1],
                                        'booster': ['gbtree', 'gblinear'],
                                        'learning_rate': [0.05, 0.1, 0.15, 0.2],
                                        'max_depth': [2, 3, 5, 10, 15],
                                        'min_child_weight': [1, 2, 3, 4],
                                        'n_estimators': [100, 500, 900, 1100,
                                                         1500]},
                   random_state=42, return_train_score=True,
                   scoring='neg_mean_absolute_error', verbose=5)

In [81]:
#print out best parameters
print("Best hyperparameters:", random_cv.best_params_)

Best hyperparameters: {'n_estimators': 900, 'min_child_weight': 1, 'max_depth': 2, 'learning_rate': 0.1, 'booster': 'gbtree', 'base_score': 0.25}


In [241]:
#get best estimator and fit it to the data
best = random_cv.best_estimator_
best.fit(X_train,y_train)

XGBRegressor(base_score=0.25, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=2, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=900, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [242]:
#test_df["SalePrice"] = best.predict(X_test)
#test_df[["Id","SalePrice"]].to_csv("prediction1.csv", index=False)

In [243]:
#debug cell: check if the shape is correct
print(X_test.shape)
print(X_train.shape)
importances = best.feature_importances_
print(importances.shape)

(1459, 271)
(1460, 271)
(271,)


In [244]:
# Get the feature importances and check for high correlation
#if a high correlation is there, drop one column
import numpy as np

importances = best.feature_importances_

def drop_corr_cols(X, importances, threshold=0.9):
    cols_to_drop = []
    for i in range(X.shape[1]):
        for j in range(i+1, X.shape[1]):
            corr = np.corrcoef(X[:, i], X[:, j])[0, 1]
            if abs(corr) >= threshold:
                if importances[i] >= importances[j]:
                    cols_to_drop.append(j)
                    #print("Dropped feature", j, "because it was highly correlated with feature", i)
                else:
                    cols_to_drop.append(i)
                    #print("Dropped feature", i, "because it was highly correlated with feature", j)
    X = np.delete(X, cols_to_drop, axis=1)
    return X, cols_to_drop

X_train, cols_to_drop = drop_corr_cols(X_train, importances)

In [245]:
#debug cell for shape
print(X_test.shape)
print(X_train.shape)

(1459, 271)
(1460, 259)


In [247]:
#delete the cols that have high correlation in the test set as well
X_test = np.delete(X_test,cols_to_drop, axis=1)

In [249]:
#fit the best regressor on the new data without the highly correlated cols
best = random_cv.best_estimator_
best.fit(X_train, y_train)

XGBRegressor(base_score=0.25, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=2, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=900, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [250]:
#predict prices and make new column
test_df["SalePrice"] = best.predict(X_test)
test_df[["Id","SalePrice"]].to_csv("prediction1.csv", index=False)